In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import numpy as np
import string
from gensim import corpora
import gensim
from nltk.corpus import words
from gensim.models import LdaModel, LsiModel, Word2Vec, CoherenceModel, TfidfModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import PCA
from collections import defaultdict
from spacy.lang.en import STOP_WORDS
from pprint import pprint
import gensim.sklearn_api.ldamodel as gensimsklearn
from sklearn.manifold import TSNE
from stemming.porter2 import stem
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Topic coherence and its graph, top topics

In [3]:
# configure number of worker for multicore
import multiprocessing
cpu_count = multiprocessing.cpu_count()
worker = cpu_count - 1
print "No of worker to be used = %s"%worker

No of worker to be used = 7


In [4]:
# stop words
stop_wrd = []
with open('stopword.txt') as f:
    for c in f:
        c = c.replace(",","").replace("\n","").replace('"',"").strip()
        stop_wrd.append(c)
stop = list(STOP_WORDS)
print(len(stop))

305


# Creating Functions

In [5]:
# read document using filename
def read_document(file_name):
    print "Reading Document %s"%file_name
    temp = []
    with open(file_name) as f:
        for l in f:
            temp.append(l)
    print "Document Finished Reading"
    temp = clean_document(temp)
    return temp

In [6]:
#pls check this
def clean_document(doc_complete):
    print "Document Cleaning Started"
    doc_clean = []
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    def clean(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        stop_free = " ".join([i for i in punc_free.split() if i not in stop_wrd])
        stop_free = ' '.join(i for i in stop_free.split() if not i.startswith('u0'))
        stop_free = " ".join([i for i in stop_free.lower().split() if not unicode(i, 'utf-8').isnumeric()])
        stemm = ' '.join(stem(i) for i in stop_free.split())
        normalized = " ".join(lemma.lemmatize(word) for word in stemm.split())
        return normalized
    doc_clean = [clean(doc).split() for doc in doc_complete]
    print "Document Cleaning Ended"
    return doc_clean

In [7]:
# return token frequency , clean document (using frequency of token)
def frequency_calculate(doc_clean):
    frequency = defaultdict(int)
    for text in doc_clean:
        for token in text:
            frequency[token] += 1        
    # make new text data by removing out the token that have occurence less then 2
    minimum_freq = 2
    len_of_token = 2
    # doc_clean = [[token for token in text if frequency[token] > minimum_freq]
    #           for text in doc_clean]
    doc_clean = [[token for token in text if len(token) > len_of_token and frequency[token] > minimum_freq]
              for text in doc_clean]
    print "Total Number of distinct keywords in document after cleaning is %s"%len(frequency.keys())
    return frequency, doc_clean

# LDA (Latent Dirichlet Allocation) Model -- Based on Probabilistic Graphical Models 

<div class="alert alert-block alert-info"> <font color='Blue'> Interpreting pyLDAvis output:
<ul> <li>Bubbles on the left-hand side of plot represents a topic. Larger the bubble, the more prevalent is that topic. </li>
<li>Good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant. </li>
<li>Model with too many topics, will have many overlaps, small sized bubbles are clustered in one region of the chart.</li>
<li>The words on the right-hand side are the salient keywords that form the selected topic.</li>
<li>The red bars represent the frequency of a term in a given topic, and the blue bars represent term's frequency across the entire corpus.</li> </ul></font> </div>

In [8]:
def prepare_lda_prerequisite(doc_clean):
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    return dictionary, doc_term_matrix

In [9]:
def find_lda_optimzal_number_topics(dictionary, corpus, texts, limit=101):
    c_v = [0]*limit
    lm_list = []
    for num_topics in range(10, limit):
        lm = gensim.models.ldamulticore.LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v[num_topics] = cm.get_coherence()
        print "Coherence of model with number of topic = %s is %s"%(num_topics,cm.get_coherence())
        
    best_topic = c_v.index(max(c_v))
    print "Optimal Number of Topics in LDA model is %s with cohernce value %s"%(best_topic,max(c_v))
    return best_topic

In [10]:
def run_lda(dictionary,doc_term_matrix,num_topics=100):
    Lda = gensim.models.ldamulticore.LdaMulticore
    ldamodel = Lda(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=20, workers=worker)
    return ldamodel

# <div class="alert alert-block alert-info"><font color='red'> Non-Anonymous User (Negative Comments) <font> </div> #



In [11]:
# read the document
doc_cleaned = read_document('verified_negativecomment.txt')

Reading Document verified_negativecomment.txt
Document Finished Reading
Document Cleaning Started
Document Cleaning Ended


In [12]:
frequency_doc, doc_cleaned = frequency_calculate(doc_cleaned)

Total Number of distinct keywords in document after cleaning is 84309


In [13]:
dictionary, doc_term_matrix = prepare_lda_prerequisite(doc_cleaned)

In [14]:
num_lda_topics =  find_lda_optimzal_number_topics(dictionary,doc_term_matrix,doc_cleaned)

Coherence of model with number of topic = 10 is 0.4650348124466592
Coherence of model with number of topic = 11 is 0.46869489612356696
Coherence of model with number of topic = 12 is 0.47594738496942496
Coherence of model with number of topic = 13 is 0.4477310305931159
Coherence of model with number of topic = 14 is 0.4659642754021219
Coherence of model with number of topic = 15 is 0.45776083872758433
Coherence of model with number of topic = 16 is 0.47029336237378283
Coherence of model with number of topic = 17 is 0.44736484328168596
Coherence of model with number of topic = 18 is 0.4406028931391488
Coherence of model with number of topic = 19 is 0.46072878462509476
Coherence of model with number of topic = 20 is 0.46818213721080326
Coherence of model with number of topic = 21 is 0.4673397787108058
Coherence of model with number of topic = 22 is 0.4381743147480405
Coherence of model with number of topic = 23 is 0.44177450133348395
Coherence of model with number of topic = 24 is 0.4596

In [15]:
ldamodel = run_lda(dictionary,doc_term_matrix,num_lda_topics)

In [16]:
%time
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      12.544460        1       1 -0.012375  0.092942
6      12.160575        1       2  0.032935 -0.031128
7      11.777917        1       3  0.104145  0.046062
1       9.429457        1       4  0.113732 -0.027127
4       8.799572        1       5  0.110691 -0.003029
3       8.664100        1       6  0.144823  0.005340
0       8.572797        1       7 -0.017945  0.066520
2       7.078056        1       8  0.115195 -0.005204
8       6.855601        1       9  0.069741 -0.014345
11      5.272985        1      10 -0.179254 -0.228911
10      4.894310        1      11 -0.274112  0.265852
5       3.950173        1      12 -0.207576 -0.166970, topic_info=      Category          Freq      Term         Total  loglift  logprob
term                                                                  
15773  Default   8480.000000     money   8480.000000  30.0000  30.0000
12460  Default   4455.000000      fake   4455.000000  29.0000  29.0000
16900  Default   9312.000000   corrupt   9312.000000  28.0000  28.0000
13958  Default   4975.000000    muslim   4975.000000  27.0000  27.0000
9073   Default   4870.000000     polic   4870.000000  26.0000  26.0000
19802  Default  19028.000000     india  19028.000000  25.0000  25.0000
9601   Default   3752.000000     black   3752.000000  24.0000  24.0000
2787   Default   7160.000000     parti   7160.000000  23.0000  23.0000
14100  Default   3386.000000     price   3386.000000  22.0000  22.0000
15073  Default  10265.000000    govern  10265.000000  21.0000  21.0000
18088  Default   3780.000000     hindu   3780.000000  20.0000  20.0000
18744  Default   3332.000000       sad   3332.000000  19.0000  19.0000
23398  Default   3715.000000  pakistan   3715.000000  18.0000  18.0000
21428  Default   5935.000000       bad   5935.000000  17.0000  17.0000
5011   Default   5176.000000      case   5176.000000  16.0000  16.0000
9297   Default   6819.000000      govt   6819.000000  15.0000  15.0000
19832  Default   4924.000000       law   4924.000000  14.0000  14.0000
17539  Default   3272.000000     court   3272.000000  13.0000  13.0000
21617  Default   4791.000000    minist   4791.000000  12.0000  12.0000
12952  Default   5001.000000     illeg   5001.000000  11.0000  11.0000
12284  Default   9674.000000   countri   9674.000000  10.0000  10.0000
23952  Default   2254.000000       tax   2254.000000   9.0000   9.0000
23432  Default   5729.000000      kill   5729.000000   8.0000   8.0000
5050   Default   5649.000000     delhi   5649.000000   7.0000   7.0000
27     Default   2725.000000     china   2725.000000   6.0000   6.0000
10422  Default   3006.000000   comment   3006.000000   5.0000   5.0000
19409  Default  11841.000000    indian  11841.000000   4.0000   4.0000
18618  Default  19421.000000     peopl  19421.000000   3.0000   3.0000
12156  Default   2741.000000    easier   2741.000000   2.0000   2.0000
3283   Default   2115.000000     ignor   2115.000000   1.0000   1.0000
...        ...           ...       ...           ...      ...      ...
13111  Topic12    451.342438     tomar    505.463806   3.1182  -5.1760
20416  Topic12    153.670135      nurs    160.489059   3.1880  -6.2535
11665  Topic12    149.011520      boko    156.238739   3.1840  -6.2842
21550  Topic12    245.264679     super    276.901306   3.1101  -5.7859
4599   Topic12    193.088394      dupe    210.762283   3.1438  -6.0251
686    Topic12    300.175934      rush    360.076813   3.0495  -5.5839
21017  Topic12   1231.467896     crore   2232.856934   2.6363  -4.1723
22492  Topic12    414.925018     stock    566.606079   2.9198  -5.2602
9402   Topic12    273.863464     tenur    351.147888   2.9828  -5.6756
2898   Topic12    164.266434       amp    179.422043   3.1432  -6.1868
21556  Topic12    294.960938   assault    420.230988   2.8774  -5.6014
16348  Topic12    773.665710   account   1735.976929   2.4232  -4.6371
18135 

In [ ]:
# Topic 1: Politics(corruption, scam) 
# Topic 2: Education and job (food, less jobs)
# Topic 3: Politics (Blame, fool, stupid, idiot, fool)
# Topic 4: Accident (Phone, car, danger, fall)
# Topic 5: Border issues (army, terrorist, kill, attack, war)
# Topic 6: Communal disharmony (secular, kill, hate, convert)
# Topic 7: Law and Order (Police, arrest, criminal, punish, jail, illegal)
# Topic 8: Crime (kill, murder, death, punish)
# Topic 9: Economy and politics (bad, wrong, criticise)
# Topic 10: Black money (tax, jail, income, interest)
# Topic 11: Corruption (lie, price, inflation)
# Topic 12: Government policies and law (fake, useless, failure, resign, murder)

# <div class="alert alert-block alert-info"><font color='red'> Anonymous user (Negative Comments) </font></div>

In [17]:
# read the document
doc_cleaned = read_document('anony_negativecomment.txt')

Reading Document anony_negativecomment.txt
Document Finished Reading
Document Cleaning Started
Document Cleaning Ended


In [18]:
frequency_doc, doc_cleaned = frequency_calculate(doc_cleaned)

Total Number of distinct keywords in document after cleaning is 704


In [19]:
dictionary, doc_term_matrix = prepare_lda_prerequisite(doc_cleaned)

In [20]:
num_lda_topics =  find_lda_optimzal_number_topics(dictionary,doc_term_matrix,doc_cleaned)

Coherence of model with number of topic = 10 is 0.47041144585314515
Coherence of model with number of topic = 11 is 0.4752156096177307
Coherence of model with number of topic = 12 is 0.4652309033891455
Coherence of model with number of topic = 13 is 0.47096174824640663
Coherence of model with number of topic = 14 is 0.4672498912320216
Coherence of model with number of topic = 15 is 0.4609842425494831
Coherence of model with number of topic = 16 is 0.5013875116047704
Coherence of model with number of topic = 17 is 0.4997698082706584
Coherence of model with number of topic = 18 is 0.4910470934573593
Coherence of model with number of topic = 19 is 0.5519267309991983
Coherence of model with number of topic = 20 is 0.5234043543468954
Coherence of model with number of topic = 21 is 0.4718209901024699
Coherence of model with number of topic = 22 is 0.44000539847202447
Coherence of model with number of topic = 23 is 0.5037170512879208
Coherence of model with number of topic = 24 is 0.436161598

In [21]:
ldamodel = run_lda(dictionary,doc_term_matrix,num_lda_topics)

In [22]:
%time
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      19.012819        1       1  0.177643 -0.237052
11     10.887670        1       2 -0.252810 -0.051296
1       8.966438        1       3 -0.007696  0.050257
14      8.124160        1       4 -0.036085  0.068866
10      7.158713        1       5 -0.073281  0.027230
9       6.871081        1       6 -0.161393 -0.106042
7       6.630948        1       7  0.114250  0.054638
3       5.568451        1       8 -0.046588  0.123465
12      5.011175        1       9  0.059291  0.196347
6       4.251077        1      10  0.096259  0.026572
2       4.096206        1      11  0.076588  0.121838
16      3.730353        1      12  0.049121 -0.102371
15      2.583280        1      13  0.123388 -0.002849
8       2.195973        1      14  0.024543 -0.125886
0       1.926099        1      15 -0.049332 -0.015149
17      1.140192        1      16 -0.039467 -0.000695
4       0.886127        1      17 -0.036698 -0.011556
13      0.479623        1      18 -0.008867 -0.008159
18      0.479623        1      19 -0.008867 -0.008159, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
34    Default   6.000000       judg   6.000000  30.0000  30.0000
71    Default   8.000000      peopl   8.000000  29.0000  29.0000
115   Default   3.000000       time   3.000000  28.0000  28.0000
22    Default  10.000000     govern  10.000000  27.0000  27.0000
73    Default   7.000000      child   7.000000  26.0000  26.0000
11    Default   5.000000     prison   5.000000  25.0000  25.0000
7     Default   5.000000      islam   5.000000  24.0000  24.0000
10    Default   5.000000     articl   5.000000  23.0000  23.0000
96    Default   6.000000      parti   6.000000  22.0000  22.0000
30    Default   8.000000    husband   8.000000  21.0000  21.0000
64    Default   3.000000      hindu   3.000000  20.0000  20.0000
86    Default   8.000000      india   8.000000  19.0000  19.0000
83    Default   7.000000    employe   7.000000  18.0000  18.0000
13    Default   8.000000      local   8.000000  17.0000  17.0000
55    Default   5.000000   militari   5.000000  16.0000  16.0000
44    Default   3.000000      state   3.000000  15.0000  15.0000
19    Default   5.000000      delhi   5.000000  14.0000  14.0000
99    Default   3.000000      polit   3.000000  13.0000  13.0000
104   Default   6.000000      wrong   6.000000  12.0000  12.0000
33    Default   4.000000  christian   4.000000  11.0000  11.0000
114   Default   3.000000       jail   3.000000  10.0000  10.0000
66    Default   2.000000     system   2.000000   9.0000   9.0000
37    Default   4.000000    respons   4.000000   8.0000   8.0000
97    Default   4.000000      split   4.000000   7.0000   7.0000
108   Default   4.000000    develop   4.000000   6.0000   6.0000
67    Default   4.000000       citi   4.000000   5.0000   5.0000
69    Default   7.000000       call   7.000000   4.0000   4.0000
57    Default   7.000000    countri   7.000000   3.0000   3.0000
23    Default   4.000000       govt   4.000000   2.0000   2.0000
110   Default   4.000000     villag   4.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
31    Topic19   0.020169       mean   3.234171   0.2625  -4.7622
72    Topic19   0.020169       hurt   3.235605   0.2621  -4.7622
70    Topic19   0.020169       head   3.264718   0.2531  -4.7622
63    Topic19   0.020169      prove   3.265440   0.2529  -4.7622
77    Topic19   0.020169        car   3.273413   0.2505  -4.7622
35    Topic19   0.020169       seat   3.273413   0.2505  -4.7622
111   Topic19   0.020169      clear   3.274905   0.2500  -4.7622
32    Topic19   0.020169    societi   3.275446   0.2499  -4.7622
0     Topic19   0.020169     forget   3.281080   0.2481  -4.7622
42    Topic19   0.020169     poison   3.291545   0.2449  -4.7622
30    Topic19   0.020169    

In [ ]:
# Topic 1: Politics (government, vote, party, wrong, case)
# Topic 2: Workplace (Employee, quit, penalty, death)
# Topic 3: Crime (man poisons woman)
# Topic 4: Crime (husband throws wife, money, treason, court)
# Topic 5: Politics (vote, blame, wrong)
# Topic 6: Politics (fail, vote, treason)
# Topic 7: Politics (hurt, fail, death)
# Topic 8: Accident (car accident, death)
# Topic 9: Crime (prison, money, penalty)
# Topic 10: Crime (child exploitation)
# Topic 11: Judiciary (money, treason, penalty)
# Topic 12: Crime (penalty, poison, throw, condition)
# Topic 13: Workplace (fail, quit, treason, money)
# Topic 14: Politics (Government, fail, penalty)
# Topic 15: Crime (money, court, penalty)
# Topic 16: Crime (court, penalty, hurt, doubt)
# Topic 17: Crime (court, hurt, prove, fail)
# Topic 18: Crime (money, court, fail, limit)

# <div class="alert alert-block alert-info"><font color='red'> Users with Anonymous profiles (Negative Comments) </font> </div>

In [23]:
# read the document
doc_cleaned = read_document('null_negativecomment.txt')

Reading Document null_negativecomment.txt
Document Finished Reading
Document Cleaning Started
Document Cleaning Ended


In [24]:
frequency_doc, doc_cleaned = frequency_calculate(doc_cleaned)

Total Number of distinct keywords in document after cleaning is 6991


In [25]:
dictionary, doc_term_matrix = prepare_lda_prerequisite(doc_cleaned)

In [26]:
num_lda_topics =  find_lda_optimzal_number_topics(dictionary,doc_term_matrix,doc_cleaned)

Coherence of model with number of topic = 10 is 0.2893470678649528
Coherence of model with number of topic = 11 is 0.3210477087750558
Coherence of model with number of topic = 12 is 0.31533488906393614
Coherence of model with number of topic = 13 is 0.30531871517252135
Coherence of model with number of topic = 14 is 0.3025408823444958
Coherence of model with number of topic = 15 is 0.30805440917313415
Coherence of model with number of topic = 16 is 0.28828237070985735
Coherence of model with number of topic = 17 is 0.31287576581548016
Coherence of model with number of topic = 18 is 0.2866261226860264
Coherence of model with number of topic = 19 is 0.3050257289883424
Coherence of model with number of topic = 20 is 0.30429593895076607
Coherence of model with number of topic = 21 is 0.2960015710613187
Coherence of model with number of topic = 22 is 0.3117491042248529
Coherence of model with number of topic = 23 is 0.31009585656029026
Coherence of model with number of topic = 24 is 0.30843

In [27]:
ldamodel = run_lda(dictionary,doc_term_matrix,num_lda_topics)

In [28]:
%time
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
50     7.337613        1       1 -0.076424 -0.376212
43     2.802343        1       2 -0.032497 -0.058200
22     2.416811        1       3  0.053654  0.028394
2      2.271334        1       4 -0.038396 -0.037258
3      2.113068        1       5 -0.013843 -0.165308
79     2.045402        1       6  0.144453 -0.029881
10     1.952614        1       7  0.012408 -0.010611
75     1.807790        1       8  0.029043 -0.053825
29     1.759089        1       9  0.106137 -0.050926
11     1.727893        1      10  0.106278  0.043594
30     1.600177        1      11 -0.034932 -0.105803
84     1.533548        1      12  0.027953  0.032050
28     1.515920        1      13 -0.095486 -0.023160
59     1.511048        1      14  0.129129  0.025237
14     1.492785        1      15 -0.062542 -0.058318
73     1.475346        1      16 -0.006023  0.061717
76     1.433577        1      17 -0.089361  0.025620
56     1.407313        1      18  0.108490 -0.003646
32     1.372748        1      19  0.041098 -0.062126
33     1.302274        1      20  0.058637 -0.109996
83     1.292328        1      21  0.047772 -0.061230
66     1.217716        1      22  0.041957  0.069440
47     1.214614        1      23 -0.037933  0.069720
78     1.177445        1      24 -0.073947  0.027211
61     1.151350        1      25 -0.053727  0.065305
15     1.148602        1      26  0.085488 -0.039344
7      1.145043        1      27  0.152884 -0.092580
54     1.110559        1      28 -0.016082 -0.028493
38     1.107938        1      29  0.026631  0.077445
67     1.085079        1      30 -0.092923 -0.046837
...         ...      ...     ...       ...       ...
57     0.887138        1      57  0.027777  0.030993
49     0.877822        1      58 -0.006069  0.073218
12     0.877013        1      59  0.010148  0.015874
48     0.866521        1      60  0.074285  0.071081
17     0.853184        1      61 -0.040857 -0.055987
23     0.842401        1      62  0.130012 -0.015049
51     0.818666        1      63 -0.031782  0.059968
41     0.815316        1      64 -0.001084  0.043592
36     0.814752        1      65 -0.041986  0.017714
81     0.813450        1      66  0.080198 -0.016258
13     0.812592        1      67 -0.030324  0.061555
18     0.812415        1      68 -0.047188  0.034233
46     0.803829        1      69  0.001884  0.024384
20     0.800309        1      70 -0.056250 -0.023438
85     0.799160        1      71 -0.026574  0.083261
31     0.758413        1      72 -0.065409  0.010341
27     0.747765        1      73 -0.049481  0.016576
64     0.734794        1      74 -0.029270 -0.038348
71     0.716027        1      75 -0.023864 -0.057552
68     0.711830        1      76  0.047791  0.014147
34     0.695904        1      77  0.047760  0.053639
82     0.669047        1      78 -0.060597  0.040241
55     0.660634        1      79 -0.024129  0.016205
42     0.657227        1      80 -0.041084  0.025998
72     0.621300        1      81 -0.034645  0.047890
26     0.593551        1      82 -0.046234  0.047680
70     0.575668        1      83 -0.054527  0.069237
39     0.559035        1      84 -0.083793 -0.001166
65     0.529770        1      85 -0.070209 -0.058596
40     0.468563        1      86 -0.019910  0.020363

[86 rows x 5 columns], topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
839   Default  243.000000  antislaught  243.000000  30.0000  30.0000
59    Default  246.000000         cull  246.000000  29.0000  29.0000
1934  Default  245.000000     recognis  245.000000  28.0000  28.0000
774   Default  246.000000    donu2019t  246.000000  27.0000  27.0000
84    Default  246.000000       permit  246.000000  26.0000  26.0000
677   Default  252.000000      realiti  252.000000  25.0000  25.0000
1451  Default  249.000000       select  249

In [ ]:
# Topic 1: Law (anti-slaughter cow law, reality)
# Topic 2: Economy (cash, cashless, corruption)
# Topic 3: Female exploitation (patriarch society)
# Topic 4: Colonisation (exploitation of people)
# Topic 5: Corruption (govt. staff, ombudsman, notice, investigation)
# Topic 6: Border water issues
# Topic 7: Black money (list, Supreme Court)
# Topic 8: Economy (Corruption, infrastructure, failure)
# Topic 9: School (police, boy, girl, friend, shoots)
# Topic 10: Politics (hide, poor, shame, faith, coward) 
# Topic 11: Corporate politics (land, farmer)
# Topic 12: Moral issues in kid (blast, bomb, lie, problem)
# Topic 13: Law (summon, court, law)
# Topic 14: Politics (vote, stupid, foolish, Encompass, corrupt)
# Topic 15: Politics (two-faced politician, real face, ugly, pretend)
# Topic 16: Work (money, hate work, watch, fun)
# Topic 17: Sports (team, hardwork, time left)
# Topic 18: Politics (power, conspiracy, poor hungry ppl, failure)